<a href="https://colab.research.google.com/github/ddinesan/Manga/blob/master/Lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Software for Quantum Computing**

This is a worksheet prepared by Evan Peters for Lecture 6 of the reading course "Introduction to Quantum Computer Programming" (AMATH 900/ AMATH 495/ QIC 895) at the University of Waterloo.

Course Webpage: https://sites.google.com/view/quantum-computer-programming

Text followed in the course: [Quantum Computing, An Applied Approach](https://www.springer.com/gp/book/9783030239213) by Jack D. Hidary (2019)

In [ ]:
!pip install cirq qiskit pyquil

# Quantum Computing Software

In the lecture, we discussed abstractions of how software interfaces with quantum computers, as well as the need for extra layers of abstraction to account for high error rates in physical qubits. The goal of this notebook is the following:
1. Demo some python quantum computing libraries
2. Implement a logical qubit via Shor's code

## Quantum computing libraries in Python

## Qiskit

Lets look at a sample program in Qiskit




In [ ]:
import qiskit

# Create a quantum register with one qubit
qreg = qiskit.QuantumRegister(1, name='qreg')
# Create a classical register with one qubit
creg = qiskit.ClassicalRegister(1, name='creg')

# Qubits in a circuit (registers) must be explicitly declared at circuit creation
circuit = qiskit.QuantumCircuit(qreg, creg)

# Add a Rx operation on the qubit
circuit.rx(np.pi/2, qreg[0])
# Print the circuit
print(circuit.draw())

           ┌──────────┐
qreg_0: |0>┤ Rx(pi/2) ├
           └──────────┘
 creg_0: 0 ════════════
                       


In [ ]:
# There are multiple backends available to run on
for backend in qiskit.BasicAer.backends():
  print(backend)

# For state-vector simulation:
backend = qiskit.BasicAer.get_backend("statevector_simulator")
job = qiskit.execute(circuit, backend)
print("State vector:")
print(job.result().get_statevector())

# For measurement-based simulation: We must add a measurement on the qubit
circuit.measure(qreg, creg)
backend = qiskit.BasicAer.get_backend("qasm_simulator")
job = qiskit.execute(circuit, backend, shots=100)
print("Measurements:")
print(job.result().get_counts())

display(job.result())

qasm_simulator
statevector_simulator
unitary_simulator
State vector:
[1.+0.j 0.+0.j]
Measurements:
{'1': 50, '0': 50}


Result(backend_name='qasm_simulator', backend_version='2.0.0', header=Obj(backend_name='qasm_simulator', backend_version='2.0.0'), job_id='3bcaca73-ba8f-427b-8d4f-e7826eb70063', qobj_id='3f3c5bc1-f886-4fff-b20a-dd7d0d99b67f', results=[ExperimentResult(data=ExperimentResultData(counts=Obj(0x0=50, 0x1=50)), header=Obj(clbit_labels=[['creg', 0]], creg_sizes=[['creg', 1]], memory_slots=1, n_qubits=1, name='circuit31', qreg_sizes=[['qreg', 1]], qubit_labels=[['qreg', 0]]), meas_level=<MeasLevel.CLASSIFIED: 2>, name='circuit31', seed_simulator=691107222, shots=100, status='DONE', success=True, time_taken=0.0011074542999267578)], status='COMPLETED', success=True, time_taken=0.0011513233184814453)

### Cirq

You should be familiar with Cirq by now. Instead of reviewing Cirq's code, lets look closer at how Cirq's software-hardware hierarchy is supposed to work.

Cirq doesn't offer compilation into its own assembly language, but it _does_ support compilation to QASM:

In [ ]:
import cirq

circuit = cirq.Circuit()
q0, q1 = cirq.GridQubit.rect(1, 2)

circuit += cirq.rx(np.pi/2).on(q0)
circuit += cirq.FSimGate(np.pi/2, np.pi/6).on(q0, q1)

# Convert this simple circuit into QASM
qasm_circuit = cirq.qasm(circuit)
print(qasm_circuit.replace("\\n", "\n")) # unescape special characters


// Generated from Cirq v0.7.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [(0, 0), (0, 1)]
qreg q[2];


rx(pi*0.5) q[0];

// Gate: cirq.FSimGate(theta=1.5707963267948966, phi=0.5235987755982988)
u3(0,0,pi*1.0) q[0];
u3(0,0,pi*1.0) q[1];
rx(pi*0.5) q[0];
cx q[0],q[1];
rx(0) q[0];
ry(pi*0.5) q[1];
cx q[1],q[0];
rx(pi*-0.5) q[1];
rz(pi*0.5) q[1];
cx q[0],q[1];
u3(pi*1.0,0,0) q[0];
u3(pi*1.0,0,0) q[1];
u3(0,pi*1.0,pi*0.5) q[0];
u3(0,pi*1.0,pi*0.5) q[1];
rx(pi*0.5) q[0];
cx q[0],q[1];
rx(0) q[0];
ry(pi*0.5) q[1];
cx q[1],q[0];
rx(pi*-0.5) q[1];
rz(pi*0.5) q[1];
cx q[0],q[1];
u3(pi*1.0,0,pi*0.5) q[0];
u3(pi*1.0,0,pi*0.5) q[1];
u3(pi*0.5,pi*1.0,pi*0.5878634621) q[0];
u3(pi*0.5,0,pi*0.0878634621) q[1];
rx(pi*0.5) q[0];
cx q[0],q[1];
rx(pi*0.4166666667) q[0];
ry(pi*0.5) q[1];
cx q[1],q[0];
rx(pi*-0.5) q[1];
rz(pi*0.5) q[1];
cx q[0],q[1];
u3(pi*0.5,pi*0.3288032046,0) q[0];
u3(pi*0.5,pi*0.8288032046,pi*1.0) q[1];



**Do you notice anything odd about that program? Whats going on here?**

---
*Code Exercise*

Try to determine IBM's native two-qubit gates. What are u1, u2, u3?

(Answer at end of worksheet)


In [ ]:
# Your code here
def qasm_pretty(s):
  return s.replace("\\n", "\n")

print(qasm_pretty(cirq.qasm(cirq.Circuit(cirq.CNOT(q0, q1)))))
print(qasm_pretty(cirq.qasm(cirq.Circuit(cirq.ISWAP(q0, q1)))))

// Generated from Cirq v0.7.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [(0, 0), (0, 1)]
qreg q[2];


cx q[0],q[1];

// Generated from Cirq v0.7.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [(0, 0), (0, 1)]
qreg q[2];


// Gate: ISWAP
cx q[0],q[1];
h q[0];
cx q[1],q[0];
rz(pi*0.5) q[0];
cx q[1],q[0];
rz(pi*-0.5) q[0];
h q[0];
cx q[0],q[1];



---

In the lecture we saw that Google's software hierarchy seemed to be missing a compilation step. Instead of explicit compilation to a new language, Cirq programs are likely serialized and passed into a control device that can parse the serial circuit description for hardware control details.

We can look at an example of this:

In [ ]:
print(circuit, "\n")
cirq.google.SYC_GATESET.serialize(circuit)

(0, 0): ───Rx(0.5π)───fsim(0.5π, 0.167π)───
                      │
(0, 1): ──────────────#2─────────────────── 



language {
  gate_set: "sycamore"
}
circuit {
  scheduling_strategy: MOMENT_BY_MOMENT
  moments {
    operations {
      gate {
        id: "xy"
      }
      args {
        key: "axis_half_turns"
        value {
          arg_value {
            float_value: 0.0
          }
        }
      }
      args {
        key: "half_turns"
        value {
          arg_value {
            float_value: 0.5
          }
        }
      }
      qubits {
        id: "0_0"
      }
    }
  }
  moments {
    operations {
      gate {
        id: "syc"
      }
      qubits {
        id: "0_0"
      }
      qubits {
        id: "0_1"
      }
    }
  }
}

### Pyquil

Pyquil is Rigetti's quantum computing library written in Python. It features similar functionality as Cirq and Qiskit, but has a slightly different process flow.

PyQuil/QUIL interface with a QVM (Quantum Virtual Machine), which is basically a quantum simulator (or hardware!) that also simulates the user interface with a quantum computer. Unfortunately, this means we can't actually run programs written in pyquil without also downloading the QVM, and so the following code will not work in Colaboratory:

In [ ]:
import pyquil
# Create a quantum program
program = pyquil.Program()

# As with qiskit, classical memory must be allocated in advance
creg = program.declare("ro", memory_type="BIT", memory_size=1)

# Make a simple program
program += [
  pyquil.gates.X(0),
  pyquil.gates.MEASURE(0, creg[0]) # Measurement will go into classical register
]
print("Program:")
print(program)

# # Get a 'quantum computer' to run on. QVM refers to "quantum virtual machine"
# and basically functions as a quantum simulator that also simulates the interface
# with real hardware
qvm = pyquil.get_qc("1q-qvm")

# Specify number of measurements to take
program.wrap_in_numshots_loop(100)

result = qvm.run(prog)
print(result)

## Logical Qubits example: Shor's code

In the lecture we discussed "Logical qubits", which are encodings of qubits that themselves function as single qubits. Here we'll figure out how this is supposed to work.

In [ ]:
q0, q1, q2, a0, a1 = cirq.LineQubit.range(5)
shor_circuit = cirq.Circuit()
shor_circuit += cirq.CNOT(q0, q1)
shor_circuit += cirq.CNOT(q1, q2)

# SECRET ERROR BELOW:

In [ ]:
#@title
# X-ERROR ON ONE QUBIT
shor_circuit += cirq.X(q1)

In [ ]:
# Ancilla qubits will capture the error signature
shor_circuit += cirq.CNOT(q0, a0)
shor_circuit += cirq.CNOT(q1, a0)
shor_circuit += cirq.CNOT(q0, a1)
shor_circuit += cirq.CNOT(q2, a1)

shor_circuit += cirq.measure(a0, a1)
print("Error correction circuit (error hidden!)")
print(shor_circuit[0:2] + shor_circuit[3:] )

Error correction circuit (error hidden!)
          ┌──┐   ┌──┐
0: ───@─────@───────────────
      │     │
1: ───X────@┼─────@─────────
           ││     │
2: ────────X┼─────┼@────────
            │     ││
3: ─────────X─────X┼────M───
                   │    │
4: ────────────────X────M───
          └──┘   └──┘


---
_Code exercise_

Determine which error was applied to the circuit, by only measuring qubits 3 and 4

(Answer at end of worksheet)

---

In general, this code allows for a single X-error to be corrected for in the circuit using classical processing, _without observing the state of the working qubits 0-2_. This means that if we can design algorithms that manipulate the combined state of the first three qubits, the "logical qubit", we can correct errors that come up without destroying our quantum state!

Such algorithms that can manipulate logical qubits employ logical gates. There are many more complicated and efficient error correction codes, such as the "toric code" and "surface code".

## Code exercise solutions

   


### IBM's hardware gates
The hardware gateset for most IBM hardware (as of 2019) is :

 - CNOT
 - Rx(pi/2)

u1 is a "Virtual-Z" gate that is performed in software, while u2 and u3 are just Virtual Z's interspersed with one or two Rx(pi/2) respectively. See the paper [Efficient Z-Gates for Quantum Computing](https://arxiv.org/pdf/1612.00858.pdf). From a user's manual for Quantum Experience:
![](https://drive.google.com/uc?id=1POoUJwT44bag96r3JAjhqmFH-y17gJ5N) 

### Determining error in a logical qubit

In [ ]:
measurements = cirq.Simulator().run(shor_circuit)
print(measurements)

3,4=1, 0


The outcomes indicate that qubit 3 (ancilla 0) is in a state of 1, qubit 4 (ancilla 1) is in a state zero. Based on the structure of CNOT's in the circit, this is consistent with an X-gate being applied to qubit 1.

In general, you can read off the qubit where an X-error occured according to the table of ancilla readouts:

```
11 -> qubit 0
10 -> qubit 1
01 -> qubit 2
00 -> no error
```